In [63]:
import pandas as pd
import xgboost
import shap
import RDG_interventional_classification
import RDG_gaussian_classification
import numpy as np
import random
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import train_test_split

In [64]:
from sktime.classification.kernel_based import RocketClassifier
from sktime.datasets import load_gunpoint
from sktime.datasets import load_basic_motions
from sktime.datasets import load_italy_power_demand

X, y = load_basic_motions()


numpy_X = []
for i in range(0,X.shape[0]):
    numpy_X.append(X.loc[i].values[0])
X = np.asarray(numpy_X)

y = np.asarray(y)

X = (X - np.min(X))/(np.max(X)-np.min(X))
#y =  (y - np.min(y))/(np.max(y)-np.min(y))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [65]:
X.shape

(80, 100)

In [66]:
feature_names = np.arange(100)

In [67]:
y

array(['standing', 'standing', 'standing', 'standing', 'standing',
       'standing', 'standing', 'standing', 'standing', 'standing',
       'running', 'running', 'running', 'running', 'running', 'running',
       'running', 'running', 'running', 'running', 'walking', 'walking',
       'walking', 'walking', 'walking', 'walking', 'walking', 'walking',
       'walking', 'walking', 'badminton', 'badminton', 'badminton',
       'badminton', 'badminton', 'badminton', 'badminton', 'badminton',
       'badminton', 'badminton', 'standing', 'standing', 'standing',
       'standing', 'standing', 'standing', 'standing', 'standing',
       'standing', 'standing', 'running', 'running', 'running', 'running',
       'running', 'running', 'running', 'running', 'running', 'running',
       'walking', 'walking', 'walking', 'walking', 'walking', 'walking',
       'walking', 'walking', 'walking', 'walking', 'badminton',
       'badminton', 'badminton', 'badminton', 'badminton', 'badminton',
       'badmin

In [68]:
reg = RocketClassifier(num_kernels=100)
reg.fit(np.asarray(X_train), np.asarray(y_train))
clf = reg
clf.predict_proba(X_test[3].reshape(1,100))[0][0]

0.0

In [69]:
from sktime.classification.deep_learning.cnn import CNNClassifier
cnn = CNNClassifier(n_epochs=50,batch_size=4,verbose=False)  
cnn.fit(X_train, y_train)
cnn.predict_proba(X_test[0].reshape(1,-1))[0][0]
cnn.score(X_test,y_test)

2023-03-16 16:08:32.963373: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


7/7 [==============================] - 0s 4ms/step


2023-03-16 16:08:36.694707: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


0.5555555555555556

In [70]:
clf = cnn

In [71]:

cov = np.cov(X.T)
means = X.mean(axis=0)

minis = np.min(X,axis=0)
maxis = np.max(X,axis=0)

ground_truth = {}
for i in range(0,len(clf.coef_[0])):
    ground_truth[i] = np.abs(clf.coef_[0][i])
sorted_indexes = sorted(ground_truth, key=ground_truth.get,reverse=True)
sorted_indexes

In [97]:
import timeit
import random

starttime = timeit.default_timer()
print("The start time is :",starttime)
e = shap.KernelExplainer(clf.predict_proba,X[0:10])
ks_shap_values = e.shap_values(X[0:1])[0]
print("The time difference is :", timeit.default_timer() - starttime)

The start time is : 455565.823238458
3/3 [==============================] - 0s 3ms/step


  0%|          | 0/1 [00:00<?, ?it/s]

5620/5620 [==============================] - 9s 2ms/step


The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LassoLarsIC())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LassoLarsIC())

If you wish to pass a sample_weight

The time difference is : 11.126312042004429


The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LassoLarsIC())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 


In [98]:
11*80

880

In [73]:
ks_shap_values[0][0]

0.0

In [74]:
clf.predict_proba(X_test)

7/7 [==============================] - 0s 7ms/step


array([[0.12930469, 0.06467529, 0.40675634, 0.39926374],
       [0.22894658, 0.13591294, 0.23107   , 0.40407047],
       [0.1354065 , 0.0670322 , 0.39594862, 0.40161264],
       [0.36549383, 0.24357054, 0.06965395, 0.32128164],
       [0.42653784, 0.37086454, 0.01209216, 0.1905054 ],
       [0.4022056 , 0.46713105, 0.0052073 , 0.12545612],
       [0.42776868, 0.42714763, 0.00589289, 0.13919076],
       [0.32462612, 0.15643947, 0.12985407, 0.38908035],
       [0.2133776 , 0.12116403, 0.2584299 , 0.40702847],
       [0.47153586, 0.44414082, 0.00079451, 0.08352879],
       [0.45809188, 0.4281269 , 0.00225016, 0.111531  ],
       [0.2518848 , 0.13926296, 0.20146671, 0.4073855 ],
       [0.41267207, 0.3971475 , 0.01344299, 0.17673746],
       [0.41842765, 0.39291167, 0.01306484, 0.17559592],
       [0.4471933 , 0.4804587 , 0.00072042, 0.07162759],
       [0.44543025, 0.4106921 , 0.00503932, 0.13883837],
       [0.41528755, 0.3585908 , 0.01761309, 0.20850854],
       [0.13477892, 0.06718227,

In [94]:
starttime = timeit.default_timer()
print("The start time is :",starttime)
x = X_test[0]
#cond_seps,cond_nonseps = RDG_gaussian_classification.RDG_separate(clf.predict_proba,x,means,cov,minis,maxis,0.5,X_train)
int_seps,int_nonseps = RDG_interventional_classification.RDG_separate(clf.predict_proba,x,means,minis,maxis,0.2)
print("The time difference is :", timeit.default_timer() - starttime)
starttime = timeit.default_timer()
print("The start time is :",starttime)
for a in range(0,1):
    ss_i_atts= RDG_interventional_classification.atts_separate(clf.predict_proba,x,means,minis,maxis,0.2,int_seps,int_nonseps)
    #ss_g_atts = RDG_gaussian_classification.atts_separate(clf.predict_proba,x,means,cov,minis,maxis,0.2,X_train,cond_seps,cond_nonseps)
print("The time difference is :", timeit.default_timer() - starttime)

The start time is : 455062.235383416
1/1 [==============================] - 0s 9ms/step


1/1 [==============================] - 0s 8ms/step


1/1 [==============================] - 0s 9ms/step
The time difference is : 10.346591833978891
The start time is : 455072.582021
1/1 [==============================] - 0s 9ms/step


1/1 [==============================] - 0s 10ms/step
The time difference is : 5.072842249996029


In [95]:
5*80 + 10.286996666982304

410.2869966669823

In [88]:
overall_ss_g_sum_indexes = []
overall_ss_i_sum_indexes = []
overall_ks_sum_indexes = []

overall_tree_sum_indexes = []


overall_ss_g_indexes = []
overall_ss_i_indexes = []
overall_ks_indexes = []

overall_tree_indexes = []

ss_g_scores = []
ss_i_scores = []
ks_scores = []
og_scores = []
#tree_scores = []

x = X_test[0]
#cond_seps,cond_nonseps = RDG_gaussian_classification.RDG_separate(clf.predict_proba,x,means,cov,minis,maxis,0.0004,X_train)
#int_seps,int_nonseps = RDG_interventional_classification.RDG_separate(clf.predict_proba,x,means,minis,maxis,0.001)

print(int_nonseps)
print(cond_nonseps)
for a in range(0,80):
    x = X[a]
    ss_i_atts= RDG_interventional_classification.atts_separate(clf.predict_proba,x,means,minis,maxis,0.2,int_seps,int_nonseps)
    ss_g_atts = RDG_gaussian_classification.atts_separate(clf.predict_proba,x,means,cov,minis,maxis,0.,X_train,cond_seps,cond_nonseps)
    
    ks_atts = {}
    for val in range(0,len(ks_shap_values[a])):
        ks_atts[val] = ks_shap_values[a][val]
    
        
    #tree_atts = {}
    #for val in range(0,len(tree_shap_values[a])):
        #tree_atts[val] = tree_shap_values[a][val]

    print(ks_atts)
    temp = {}
    for u in ss_i_atts.keys():
        temp[u] = np.abs(ss_i_atts[u])
    sorted_ss_i_indexes = sorted(temp, key=temp.get, reverse=True)
    
    temp = {}
    for u in ss_g_atts.keys():
        temp[u] = np.abs(ss_g_atts[u]) 
    sorted_ss_g_indexes = sorted(temp, key=temp.get, reverse=True)
    
    temp = {}
    for u in list(ks_atts.keys()):
        temp[u] = np.abs(ks_atts[u]) 
    print(temp)
    sorted_ks_indexes = sorted(temp, key=temp.get, reverse=True)
    
    
    #temp = {}
    #for u in tree_atts.keys():
       # temp[u] = np.abs(tree_atts[u]) 
    #sorted_tree_indexes = sorted(temp, key=temp.get, reverse=True)
    
    overall_ss_g_indexes.append(sorted_ss_g_indexes[0])
    overall_ss_i_indexes.append(sorted_ss_i_indexes[0])
    overall_ks_indexes.append(sorted_ks_indexes[0])
    #overall_og_indexes.append(sorted_og_indexes[0])
    #overall_tree_indexes.append(sorted_tree_indexes[0])
    
    total_sum = clf.predict_proba(x.reshape(1,-1))[0][0] -  clf.predict_proba(means.reshape(1,-1))[0][0]

    overall_ss_i_sum_indexes.append(sum(ss_i_atts.values()) - total_sum)
    overall_ss_g_sum_indexes.append(sum(ss_g_atts.values()) - total_sum)
    overall_ks_sum_indexes.append(sum(ks_atts.values()) - total_sum)
    #overall_og_sum_indexes.append(sum(og_atts.values()) - total_sum)
    
    
    ind = sorted_ss_g_indexes[0]
    new_x = x.copy()
    if len(list(ind)) > 1:
        for u in list(ind):
            new_x[u] = means[u]
    else:
        new_x[list(ind)[0]] = means[list(ind)[0]]
    
    prediction = clf.predict_proba(new_x.reshape(1,-1))[0][0]
    
    ss_g_scores.append(np.abs(prediction - clf.predict_proba(means.reshape(1,-1))[0][0]))
    
    ind = sorted_ss_i_indexes[0]
    new_x = x.copy()
    if len(list(ind)) > 1:
        for u in list(ind):
            new_x[u] = means[u]
    else:
        new_x[list(ind)[0]] = means[list(ind)[0]]
    
    prediction = clf.predict_proba(new_x.reshape(1,-1))[0][0]
    
    ss_i_scores.append(np.abs(prediction - clf.predict_proba(means.reshape(1,-1))[0][0]))
    
    ind = sorted_ks_indexes[0]
    new_x = x.copy()
    
    new_x[ind] = means[ind]
    
    prediction = clf.predict_proba(new_x.reshape(1,-1))[0][0]
    
    ks_scores.append(np.abs(prediction - clf.predict_proba(means.reshape(1,-1))[0][0]))
    
    

    
    """ind = sorted_tree_indexes[0]
    new_x = x.copy()
    
    new_x[ind] = means[ind]
    
    prediction = clf.predict(new_x.reshape(1,-1))
    
    tree_scores.append(np.abs(prediction - clf.predict(means.reshape(1,-1))))"""
            

[[15, 16, 19, 28, 39, 40, 31, 37, 51, 52, 18, 20, 21, 33, 34, 42, 43, 44, 45, 46, 32, 38, 41, 47, 48, 22, 24, 25, 29, 30, 35, 36, 49, 60, 64, 68, 69, 27, 50, 53, 54, 59, 23, 56, 61, 62, 63, 65, 66, 70, 71, 72], [58, 73, 74]]
[[12, 64], [17, 18, 53, 19, 20, 48, 51, 26, 27, 31, 34, 41, 43, 45, 47, 68, 69, 75, 78, 21, 22, 23, 24, 28, 29, 30, 33, 35, 37, 38, 39, 44, 46, 49, 50, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 65, 66, 67, 70, 71, 72, 73, 76, 79, 80, 84, 86, 25, 32, 36, 52, 74, 77, 83, 85, 40, 42, 87]]
14/14 [==============================] - 0s 2ms/step


14/14 [==============================] - 0s 2ms/step
in here

14/14 [==============================] - 0s 2ms/step
in here

14/14 [==============================] - 0s 2ms/step
{0: 0.001210717083781028, 1: 0.0, 2: 0.0, 3: -0.0003486281848203816, 4: 0.0, 5: -0.0006142451371865593, 6: -0.00173644804601023, 7: -0.0018835710269848635, 8: -0.002028332438238749, 9: -0.002498866748362341, 10: -0.002784149360149966, 11: -0.001562619389834215, 12: -0.0020313369764321542, 13: -0.003558397433387548, 14: -0.0007787964893257292, 15: -0.0026710798625498777, 16: -0.0036268176967148434, 17: -0.007467459048441489, 18: -0.0056877256697879484, 19: -0.0043232649125204215, 20: -0.0038932674185899103, 21: -0.0030551541988849116, 22: -0.00426481072544817, 23: -0.003120439453817977, 24: -0.002706330352055512, 25: -0.004499364868304784, 26: -0.005989861724255367, 27: -0.0054332752989561485, 28: -0.003473467230630254, 29: -0.0035947323782210745, 30: -0.0043000939035765535, 31: -0.004960107529373043, 32: -0.0051

14/14 [==============================] - 0s 2ms/step
in here

14/14 [==============================] - 0s 2ms/step
in here

14/14 [==============================] - 0s 2ms/step
{0: 0.0, 1: -0.0003636913737841196, 2: -0.00026411436573879377, 3: -0.0003179927181792742, 4: 0.0013348899244147202, 5: -0.00047747658310392897, 6: -0.0015943929739601983, 7: -0.0016766758132507108, 8: -0.0004423686426450504, 9: -0.0018946899522719947, 10: -0.002877998367919667, 11: -0.0027245005854216625, 12: -0.003930288884120102, 13: -0.0034438744853563558, 14: -0.0026257359140270343, 15: -0.004067359619841727, 16: -0.004436068896650153, 17: -0.006820759708168383, 18: -0.0077186150803019325, 19: -0.004441848119530645, 20: -0.0032171834908070705, 21: -0.004508885623038331, 22: -0.003960163791611899, 23: -0.0034299487297043984, 24: -0.004186370470253134, 25: -0.005095050635219168, 26: -0.007485358667756366, 27: -0.006025383737415441, 28: -0.003987738199777979, 29: -0.00361308521780541, 30: -0.003660078756420057

14/14 [==============================] - 0s 2ms/step


14/14 [==============================] - 0s 2ms/step
in here

14/14 [==============================] - 0s 2ms/step
in here

14/14 [==============================] - 0s 2ms/step
{0: 0.0, 1: 0.0, 2: 0.0, 3: -0.0004520314009559235, 4: -0.00021766917891867144, 5: 0.0, 6: -0.0014785284887142275, 7: -0.0011577949994075751, 8: -0.0021438613152367793, 9: -0.0026804799468279805, 10: -0.002633721148830056, 11: -0.0025527388386609763, 12: -0.0033644456656779804, 13: -0.0032869090939696027, 14: -0.0020613643417182782, 15: -0.002735507026530299, 16: -0.005011971572712108, 17: -0.006527255407293096, 18: -0.0047533915855241405, 19: -0.004488579447536946, 20: -0.0041586376170774835, 21: -0.004002933119274057, 22: -0.004175255181087659, 23: -0.0024302483296752293, 24: -0.004696771232688259, 25: -0.004849912066177115, 26: -0.006845665977459376, 27: -0.004737481966087704, 28: -0.00314551358380452, 29: -0.002698313786744319, 30: -0.0034199421955779935, 31: -0.003950611718769465, 32: -0.003833330574300496,

...

14/14 [==============================] - 0s 3ms/step
in here

14/14 [==============================] - 0s 2ms/step
in here

14/14 [==============================] - 0s 2ms/step
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: -0.00020560444873164973, 7: 0.0, 8: 0.001047104309748401, 9: 0.0018304699015049863, 10: 0.0024530224790810934, 11: 0.0018571420676094397, 12: 0.0026919358019522614, 13: 0.008419606351224438, 14: 0.0026802385133945336, 15: -0.003280212653826458, 16: -0.001627003959704905, 17: -0.0021625329224044614, 18: -0.0019409160679185176, 19: -0.0024843082840609355, 20: 0.0023169661125922937, 21: 0.0, 22: -0.002058788614562182, 23: 0.0018147174453284494, 24: 0.0011261567056875153, 25: -0.001824817928522096, 26: -0.002163372696051129, 27: 0.0, 28: 0.0004612608806026099, 29: 0.0005713144023369096, 30: -0.001244242584944995, 31: -0.0013431142098096077, 32: -0.0013334173423731211, 33: -0.0010518603517630874, 34: 0.0, 35: -0.003946716409582035, 36: -0.0034494000856789774, 37: -0

1/1 [==============================] - 0s 8ms/step


14/14 [==============================] - 0s 2ms/step
in here

14/14 [==============================] - 0s 2ms/step
in here

14/14 [==============================] - 0s 8ms/step
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: -0.00022264541754037082, 6: -0.0004441482143924466, 7: -0.00040640951291627536, 8: -0.002559199301831582, 9: -0.0032299495854974936, 10: 0.0, 11: 0.0, 12: -0.0009378805738856459, 13: -0.0010526213978288568, 14: -0.0032437382171655454, 15: 0.010657586958515403, 16: 0.010731722725572348, 17: -0.003541967754893356, 18: -0.004079673564411693, 19: -0.0036510442605070734, 20: -0.0008599481831147865, 21: -0.002337646243180974, 22: -0.0050345276143504425, 23: -0.002426465835699436, 24: -0.0006126255577541297, 25: -0.0029976771960758403, 26: -0.004586241261682139, 27: -0.003103014905519952, 28: 0.004422464629241604, 29: 0.005056980058070787, 30: 0.005395038081933361, 31: 0.004253350809394547, 32: 0.002721750394954557, 33: 0.011936046824481378, 34: 0.007377962872021165, 35: 0.00

14/14 [==============================] - 0s 6ms/step
in here

14/14 [==============================] - 0s 2ms/step
in here

14/14 [==============================] - 0s 8ms/step
{0: 0.0, 1: 0.0002860317586254428, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.002077127409119618, 10: 0.002385854985713729, 11: 0.0004727601432427085, 12: 0.0004541636695837986, 13: 0.0, 14: 0.002166301779357699, 15: 0.004109825072382438, 16: -0.004101222365127704, 17: -0.00532957609944798, 18: -0.0037214126114706383, 19: -0.0018718794658197507, 20: -0.0017796263183247514, 21: -0.0022655429968668275, 22: -0.0012580559574788923, 23: -0.0005695584360537985, 24: 0.0012508138972864777, 25: 0.002257215915747044, 26: 0.004826328686009906, 27: 0.0031870390581290493, 28: 0.009234983171148098, 29: -0.0015315146509454342, 30: -0.002214831552083924, 31: -0.0016392118446749253, 32: -0.0014217379953203046, 33: 0.00031495332513672286, 34: -0.0003878469665137031, 35: -0.0027639387218275566, 36: -0.00270224984

1/1 [==============================] - 0s 17ms/step


In [89]:
int_nonseps

[[15,
  16,
  19,
  28,
  39,
  40,
  31,
  37,
  51,
  52,
  18,
  20,
  21,
  33,
  34,
  42,
  43,
  44,
  45,
  46,
  32,
  38,
  41,
  47,
  48,
  22,
  24,
  25,
  29,
  30,
  35,
  36,
  49,
  60,
  64,
  68,
  69,
  27,
  50,
  53,
  54,
  59,
  23,
  56,
  61,
  62,
  63,
  65,
  66,
  70,
  71,
  72],
 [58, 73, 74]]

In [90]:
cond_nonseps

[[12, 64],
 [17,
  18,
  53,
  19,
  20,
  48,
  51,
  26,
  27,
  31,
  34,
  41,
  43,
  45,
  47,
  68,
  69,
  75,
  78,
  21,
  22,
  23,
  24,
  28,
  29,
  30,
  33,
  35,
  37,
  38,
  39,
  44,
  46,
  49,
  50,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  65,
  66,
  67,
  70,
  71,
  72,
  73,
  76,
  79,
  80,
  84,
  86,
  25,
  32,
  36,
  52,
  74,
  77,
  83,
  85,
  40,
  42,
  87]]

In [91]:
print('interventional')
print('sum')
print(np.mean(overall_ss_i_sum_indexes))
print(np.std(overall_ss_i_sum_indexes))

print('faithfulness')
print(np.mean(ss_i_scores))
print(np.std(ss_i_scores))

print('gaussian conditional')

print('sum')
print(np.mean(overall_ss_g_sum_indexes))
print(np.std(overall_ss_g_sum_indexes))

print('faithfulness')
print(np.mean(ss_g_scores))
print(np.std(ss_g_scores))

print('KS')

print('sum')
print(np.mean(overall_ks_sum_indexes))
print(np.std(overall_ks_sum_indexes))

print('faithfulness')
print(np.mean(ks_scores))
print(np.std(ks_scores))


print(" ")
print("ATTS:")

print("")

print("KS")

print(ks_atts)

print("I")

print(ss_i_atts)

print("G")

print(ss_g_atts)



interventional
sum
0.030055460147559644
0.025429620567048436
faithfulness
0.016205385
0.00878573
gaussian conditional
sum
5.345673868060112
0.00987145165756312
faithfulness
0.11840968
0.11365206
KS
sum
2.7939677238464354e-10
5.326481244649293e-09
faithfulness
0.12578918
0.10081897
 
ATTS:

KS
{0: 0.0, 1: 0.0002860317586254428, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.002077127409119618, 10: 0.002385854985713729, 11: 0.0004727601432427085, 12: 0.0004541636695837986, 13: 0.0, 14: 0.002166301779357699, 15: 0.004109825072382438, 16: -0.004101222365127704, 17: -0.00532957609944798, 18: -0.0037214126114706383, 19: -0.0018718794658197507, 20: -0.0017796263183247514, 21: -0.0022655429968668275, 22: -0.0012580559574788923, 23: -0.0005695584360537985, 24: 0.0012508138972864777, 25: 0.002257215915747044, 26: 0.004826328686009906, 27: 0.0031870390581290493, 28: 0.009234983171148098, 29: -0.0015315146509454342, 30: -0.002214831552083924, 31: -0.0016392118446749253, 32: -0.001421

In [79]:
np.mean(clf.predict_proba(X))

20/20 [==============================] - 0s 2ms/step


0.25

In [80]:
print(overall_ss_g_indexes)
print(overall_ss_i_indexes)
print(overall_ks_indexes)

[(88,), (88,), (88,), (14,), (17, 18, 53, 19, 20, 48, 51, 26, 27, 31, 34, 41, 43, 45, 47, 68, 69, 75, 78, 21, 22, 23, 24, 28, 29, 30, 33, 35, 37, 38, 39, 44, 46, 49, 50, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 65, 66, 67, 70, 71, 72, 73, 76, 79, 80, 84, 86, 25, 32, 36, 52, 74, 77, 83, 85, 40, 42, 87), (13,), (13,), (92,), (15,), (17, 18, 53, 19, 20, 48, 51, 26, 27, 31, 34, 41, 43, 45, 47, 68, 69, 75, 78, 21, 22, 23, 24, 28, 29, 30, 33, 35, 37, 38, 39, 44, 46, 49, 50, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 65, 66, 67, 70, 71, 72, 73, 76, 79, 80, 84, 86, 25, 32, 36, 52, 74, 77, 83, 85, 40, 42, 87)]
[(15, 16, 19, 28, 39, 40, 31, 37, 51, 52, 18, 20, 21, 33, 34, 42, 43, 44, 45, 46, 32, 38, 41, 47, 48, 22, 24, 25, 29, 30, 35, 36, 49, 60, 64, 68, 69, 27, 50, 53, 54, 59, 23, 56, 61, 62, 63, 65, 66, 70, 71, 72), (15, 16, 19, 28, 39, 40, 31, 37, 51, 52, 18, 20, 21, 33, 34, 42, 43, 44, 45, 46, 32, 38, 41, 47, 48, 22, 24, 25, 29, 30, 35, 36, 49, 60, 64, 68, 69, 27, 50, 53, 54, 59, 23, 56, 61, 62,

In [81]:
selected_x = x.copy()

In [82]:
selected_x

array([0.46138458, 0.46138458, 0.55213119, 0.67029336, 0.64832176,
       0.38384283, 0.14228985, 0.13090151, 0.74318716, 0.84503861,
       0.84503861, 0.67490715, 0.70325497, 0.70325497, 0.58759272,
       0.25434984, 0.02175254, 0.42047414, 0.67678293, 0.67678293,
       0.67214853, 0.76660412, 0.70104799, 0.58944343, 0.14917948,
       0.0832725 , 0.56184377, 0.76635648, 0.58641336, 0.62653061,
       0.6112242 , 0.651953  , 0.56164499, 0.20622513, 0.20622513,
       0.5056901 , 0.80572925, 0.71074228, 0.7229032 , 0.72830784,
       0.63752417, 0.59421633, 0.20736843, 0.20736843, 0.68517881,
       0.68517881, 0.82616056, 0.84217109, 0.79332732, 0.69804668,
       0.67559926, 0.3889821 , 0.03974241, 0.17758572, 0.66646116,
       0.79969538, 0.72971934, 0.70191608, 0.70376421, 0.66996586,
       0.44801934, 0.21403007, 0.16281669, 0.16281669, 0.74260958,
       0.74260958, 0.78232854, 0.78232854, 0.70316451, 0.77291296,
       0.20962281, 0.20962281, 0.0820307 , 0.55133596, 0.75845

In [83]:
fig, ax = plt.subplots()
x = np.arange(0, 24)

ax.plot(x, selected_x, color='black')
ax.plot(x, means, color='blue')

#threshold = 0.1
#ax.axhline(threshold, color='green', lw=2, alpha=0.7)
ax.axvspan(18, 23, alpha=0.5, color='red')

ax.axvspan(0, 4, alpha=0.5, color='green')

ax.axvspan(12, 16, alpha=0.5, color='green')
ax.set_ylabel('Power Demand')
ax.set_xlabel('Hour of Day')
plt.axvline(x=18,color='black')
plt.savefig('time_series_2.pdf')

NameError: name 'plt' is not defined

In [ ]:
clf.predict_proba(X_test[100].reshape(1,-1))

In [ ]:
clf.predict_proba(means.reshape(1,-1))